In [1]:
from newspaper import Article
import newspaper
from newspaper import fulltext
from newspaper.configuration import Configuration
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import numpy as np
import math
import time
import calendar
import json
import sqlite3

In [2]:
con = sqlite3.connect('news_urls.db')
news_url_data = pd.read_sql('select * from news_urls', con)
news_url_data = news_url_data.loc[:, ['source','topic','title','publish_date','link']]
con.close()

In [3]:
news_url_data

,source,topic,title,publish_date,link
0,yahoo.com,news,Iran vows revenge against Israel for strike on...,2021-04-12T10:11:31Z,https://news.yahoo.com/iran-vows-revenge-again...
1,yahoo.com,news,"Georgia sheriff: 3 officers wounded, 1 suspect...",2021-04-12T11:21:10Z,https://news.yahoo.com/sheriffs-official-georg...
2,yahoo.com,news,"AP PHOTOS: From Moscow to Pacific, Russia glor...",2021-04-12T07:35:18Z,https://news.yahoo.com/ap-photos-moscow-pacifi...
3,yahoo.com,news,Biden hasn't actually lifted Trump's historica...,2021-04-12T10:21:00Z,https://news.yahoo.com/biden-hasnt-actually-li...
4,yahoo.com,news,"Fiery crash kills driver, sends passenger to S...",2021-04-12T13:08:06Z,https://news.yahoo.com/fiery-crash-kills-drive...
...,...,...,...,...,...
9395,macrobusiness.com.au,economics,Daily iron ore price update (supply response),"Sun, 18 Apr 2021 23:20:08 +0000",https://www.macrobusiness.com.au/2021/04/daily...
9396,macrobusiness.com.au,economics,The “pink recession” is actually a “pink boom”,"Sun, 18 Apr 2021 23:00:22 +0000",https://www.macrobusiness.com.au/2021/04/the-p...
9397,macrobusiness.com.au,economics,Robots buy the Australian dollar,"Sun, 18 Apr 2021 14:10:41 +0000",https://www.macrobusiness.com.au/2021/04/robot...
9398,macrobusiness.com.au,economics,Capital Economics: Australian house prices to ...,"Sun, 18 Apr 2021 14:05:22 +0000",https://www.macrobusiness.com.au/2021/04/capit...


In [4]:
# news_url_data : url_data 새로 넣어온 최신버전
# news_text_data : url 업데이트 전까지의 text가 들어가 있음 --> news_url_data에만 있고, news_text_data에는 없는 것들을 target으로
con = sqlite3.connect('news_text.db')
news_text_data = pd.read_sql('select source, topic, title, publish_date, link from news_text', con)
con.close()

In [5]:
news_text_data

,source,topic,title,publish_date,link
0,yahoo.com,news,Iran vows revenge against Israel for strike on...,2021-04-12T10:11:31Z,https://news.yahoo.com/iran-vows-revenge-again...
1,yahoo.com,news,"Georgia sheriff: 3 officers wounded, 1 suspect...",2021-04-12T11:21:10Z,https://news.yahoo.com/sheriffs-official-georg...
2,yahoo.com,news,"AP PHOTOS: From Moscow to Pacific, Russia glor...",2021-04-12T07:35:18Z,https://news.yahoo.com/ap-photos-moscow-pacifi...
3,yahoo.com,news,Biden hasn't actually lifted Trump's historica...,2021-04-12T10:21:00Z,https://news.yahoo.com/biden-hasnt-actually-li...
4,yahoo.com,news,"Fiery crash kills driver, sends passenger to S...",2021-04-12T13:08:06Z,https://news.yahoo.com/fiery-crash-kills-drive...
...,...,...,...,...,...
8382,bostonglobe.com,news,Bridgette Mitchell named Northeastern women’s ...,"Sat, 17 Apr 2021 19:10:29 +0000",https://www.bostonglobe.com/2021/04/17/sports/...
8383,bostonglobe.com,news,State bans biomass power plants near environme...,"Sat, 17 Apr 2021 18:53:35 +0000",https://www.bostonglobe.com/2021/04/16/science...
8384,bostonglobe.com,news,Taking stock of the Bruins after the trade dea...,"Sat, 17 Apr 2021 18:26:23 +0000",https://www.bostonglobe.com/2021/04/17/sports/...
8385,bostonglobe.com,news,Harry and William were seen chatting together ...,"Sat, 17 Apr 2021 18:25:19 +0000",https://www.bostonglobe.com/2021/04/17/world/h...


In [6]:
# 새로 받아온 것에서 원래 db에 있는 것들을 날림
df_target = pd.merge(news_url_data, news_text_data, how='outer', indicator='Exist')
df_target = df_target.loc[df_target['Exist']=='left_only']
df_target = df_target.loc[:, ['source','topic','title','publish_date','link']]
df_target = df_target.reset_index(drop=True)
df_target

,source,topic,title,publish_date,link
0,yahoo.com,news,Police: Suspect on the run in fatal shooting o...,2021-04-18T18:15:49Z,https://news.yahoo.com/ems-3-adults-fatally-sh...
1,yahoo.com,news,'Highly unlikely' Chauvin trial ends in 'all-o...,2021-04-18T18:19:00Z,https://news.yahoo.com/highly-unlikely-chauvin...
2,yahoo.com,news,Israeli intel agencies believe Vienna talks wi...,2021-04-18T19:06:19Z,https://news.yahoo.com/israeli-intel-agencies-...
3,yahoo.com,news,DeSantis Hints at Political Consequences for C...,2021-04-18T15:58:42Z,https://news.yahoo.com/desantis-hints-politica...
4,yahoo.com,news,Suspect apprehended in fatal shooting at Wisco...,2021-04-18T10:00:55Z,https://news.yahoo.com/police-3-dead-2-wounded...
...,...,...,...,...,...
1008,macrobusiness.com.au,economics,Daily iron ore price update (supply response),"Sun, 18 Apr 2021 23:20:08 +0000",https://www.macrobusiness.com.au/2021/04/daily...
1009,macrobusiness.com.au,economics,The “pink recession” is actually a “pink boom”,"Sun, 18 Apr 2021 23:00:22 +0000",https://www.macrobusiness.com.au/2021/04/the-p...
1010,macrobusiness.com.au,economics,Robots buy the Australian dollar,"Sun, 18 Apr 2021 14:10:41 +0000",https://www.macrobusiness.com.au/2021/04/robot...
1011,macrobusiness.com.au,economics,Capital Economics: Australian house prices to ...,"Sun, 18 Apr 2021 14:05:22 +0000",https://www.macrobusiness.com.au/2021/04/capit...


In [7]:
with open('ProxyAPI.txt', 'r') as f:
    lines = f.readlines()
    api_key = lines[0].strip()

def send_request(link):
    response = requests.get(
        url="https://app.scrapingbee.com/api/v1/",
        params={
            "api_key": api_key,
            "url": link,
            "render_js": "false",
            "forward_headers": "true"
        },
        
        headers={
            "Spb-Accept-Language": "En-US",
        }
        
    )
    content = response.content
    #print('Response HTTP Status Code: ', response.status_code)
    #print('Response HTTP Response Body: ', response.content)
    return content

In [8]:
def extract_news(link):
    
    raw_html = send_request(link)
    article = Article('')
    article.download(raw_html)
    article.parse()
    article.nlp()
    keywords = article.keywords
    
    # 키워드를 리스트로 돌려주는데 그러면 db에 넣기가 곤란함
    # string으로 바꾸는데 //을 구분자로 넣을 것임
    keyword_str = ''
    for keyword in keywords:
        keyword_str = keyword_str + keyword + '//'
        
    text = article.text
    
    return keyword_str, text

In [9]:
news_text = []

for i in range(len(df_target)):
    
    print(i)
    
    try:
        # ['source','topic','title','publish_date','link']
        temp = list(np.array(df_target.loc[i].to_list()))
        
        # keyword_str, text를 temp에 append
        temp.append(extract_news(temp[4])[0])
        temp.append(extract_news(temp[4])[1])
        
        news_text.append(temp)
        
    except:
        print(temp[0], temp[1], temp[2], temp[3])
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
df_news_text = pd.DataFrame(news_text, columns=['source','topic','title','publish_date','link','keywords','text'])

In [11]:
df_news_text

,source,topic,title,publish_date,link,keywords,text
0,yahoo.com,news,Police: Suspect on the run in fatal shooting o...,2021-04-18T18:15:49Z,https://news.yahoo.com/ems-3-adults-fatally-sh...,run//broderick//shooting//car//cop//chacon//am...,"AUSTIN, Texas (AP) — A manhunt was underway Su..."
1,yahoo.com,news,'Highly unlikely' Chauvin trial ends in 'all-o...,2021-04-18T18:19:00Z,https://news.yahoo.com/highly-unlikely-chauvin...,covering//acquittal//media//chauvin//walz//hig...,Axios\n\nGov. Tim Walz (D) spoke out Sunday ov...
2,yahoo.com,news,Israeli intel agencies believe Vienna talks wi...,2021-04-18T19:06:19Z,https://news.yahoo.com/israeli-intel-agencies-...,iran//believe//meeting//intel//israeli//securi...,Israeli military intelligence and senior offic...
3,yahoo.com,news,DeSantis Hints at Political Consequences for C...,2021-04-18T15:58:42Z,https://news.yahoo.com/desantis-hints-politica...,desantis//companies//consequences//issues//wok...,Florida governor Ron DeSantis warned corporati...
4,yahoo.com,news,Suspect apprehended in fatal shooting at Wisco...,2021-04-18T10:00:55Z,https://news.yahoo.com/police-3-dead-2-wounded...,shooting//york//shotgun//wisconsin//case//flag...,The New York Times\n\nINDIANAPOLIS — They are ...
...,...,...,...,...,...,...,...
1008,macrobusiness.com.au,economics,Daily iron ore price update (supply response),"Sun, 18 Apr 2021 23:20:08 +0000",https://www.macrobusiness.com.au/2021/04/daily...,exports//supply//ore//yearangola//cutato//tonn...,The ferrous complex rallied further on April 1...
1009,macrobusiness.com.au,economics,The “pink recession” is actually a “pink boom”,"Sun, 18 Apr 2021 23:00:22 +0000",https://www.macrobusiness.com.au/2021/04/the-p...,pink//young//recession//gender//labour//actual...,One of the fakest narratives to come from the ...
1010,macrobusiness.com.au,economics,Robots buy the Australian dollar,"Sun, 18 Apr 2021 14:10:41 +0000",https://www.macrobusiness.com.au/2021/04/robot...,net//short//10yr//ust//dollar//australian//pos...,"Since February, bond market investors had gone..."
1011,macrobusiness.com.au,economics,Capital Economics: Australian house prices to ...,"Sun, 18 Apr 2021 14:05:22 +0000",https://www.macrobusiness.com.au/2021/04/capit...,fall//think//housing//strength//suggests//stab...,Capital Economics with the note:\n\nThe RBA’s ...


In [12]:
con = sqlite3.connect('news_text.db')
df_news_text.to_sql('news_text', con, if_exists='append', index_label='id')
con.close()